# Distribution of influences

To get a grip on how much randomness influences the results, we run the same model lots of times and compare the results.

In [1]:
# for Colab, install fin_benefits and unemployment-gym from Github
#!pip install -q git+https://github.com/ajtanskanen/benefits.git  
#!pip install -q git+https://github.com/ajtanskanen/econogym.git
#!pip install -q git+https://github.com/ajtanskanen/lifecycle-rl.git

# and then restart kernel
  
  # For a specific version:
#!pip install tensorflow==1.15
#!pip install stable-baselines==2.8
  
# restart kernel after running pip's

Then load all modules and set parameters for simulations.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from lifecycle_rl import Lifecycle

%matplotlib inline
%pylab inline

# varoitukset piiloon (Stable baseline ei ole vielä Tensorflow 2.0-yhteensopiva, ja Tensorflow 1.15 valittaa paljon)
# ei taida toimia piilottaminen
import warnings
warnings.filterwarnings('ignore')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Populating the interactive namespace from numpy and matplotlib


In [7]:
# parameters for the simulation
# episode = 51 / 205 timesteps (1y/3m timestep)
pop_size=10_000 # size of the population to be simulated
size1=10_000_000 #0_000 # number of timesteps in phase 1 training (callback not used)
size2=100 #0_000 # number of timesteps in phase 2 training (callback is used to save the best results)
size3=100 # number of timesteps in phase 1 training (callback not used) for policy changes
batch1=1 # size of minibatch in phase 1 as number of episodes
batch2=9_00  # size of minibatch in phase 1 as number of episodes
callback_minsteps=batch2 # how many episodes callback needs 
deterministic=False # use deterministic prediction (True) or probabilitic prediction (False)
mortality=False # include mortality in computations
randomness=True # include externally given, random state-transitions (parental leaves, disability, lay-offs) 
pinkslip=True # include lay-offs at 5 percent level each year
rlmodel='acktr' # use ACKTR algorithm
twostage=False # ajataan kahdessa vaiheessa vai ei
perusmalli='best/malli_perus3'

# Baseline

Lasketaan työllisyysasteet nykymallissa.

In [ ]:
cc1=Lifecycle(env='unemployment-v1',minimal=False,mortality=mortality,perustulo=False,
              randomness=randomness,pinkslip=pinkslip,plotdebug=False)
cc1.explain()
cc1.run_distrib(n=30,debug=False,steps1=size1,steps2=size2,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=batch1,batch2=batch2,
                save=perusmalli,plot=True,cont=True,start_from=perusmalli,results='results/distrib_base',
                callback_minsteps=callback_minsteps,rlmodel=rlmodel,twostage=twostage)

No mortality included
Parameters of lifecycle:
timestep 0.25
gamma 0.9793703613355593 (0.9200000000000003 per anno)
min_age 20
max_age 70
min_retirementage 63.5
max_retirementage 68.5
ansiopvraha_kesto300 None
ansiopvraha_kesto400 None
ansiopvraha_toe None
perustulo False
karenssi_kesto 0.25
mortality False
randomness True
include_putki None
include_pinkslip True
step 0.25

train...
phase 1
batch 1 learning rate 0.125 scaled 0.125
training...

---------------------------------
| explained_variance | 0.942    |
| fps                | 1487     |
| nupdates           | 1        |
| policy_entropy     | 0.84     |
| policy_loss        | -0.175   |
| total_timesteps    | 0        |
| value_loss         | 0.931    |
---------------------------------


---------------------------------
| explained_variance | 0.942    |
| fps                | 2555     |
| nupdates           | 10       |
| policy_entropy     | 0.784    |
| policy_loss        | 0.206    |
| total_timesteps    | 21825    |
| valu

---------------------------------
| explained_variance | 0.973    |
| fps                | 1998     |
| nupdates           | 240      |
| policy_entropy     | 0.846    |
| policy_loss        | 0.13     |
| total_timesteps    | 579575   |
| value_loss         | 0.354    |
---------------------------------
---------------------------------
| explained_variance | 0.983    |
| fps                | 1998     |
| nupdates           | 250      |
| policy_entropy     | 0.889    |
| policy_loss        | 0.0634   |
| total_timesteps    | 603825   |
| value_loss         | 0.207    |
---------------------------------
---------------------------------
| explained_variance | 0.981    |
| fps                | 1997     |
| nupdates           | 260      |
| policy_entropy     | 0.953    |
| policy_loss        | -0.029   |
| total_timesteps    | 628075   |
| value_loss         | 0.367    |
---------------------------------
---------------------------------
| explained_variance | 0.946    |
| fps         

---------------------------------
| explained_variance | 0.968    |
| fps                | 2006     |
| nupdates           | 510      |
| policy_entropy     | 0.773    |
| policy_loss        | -0.0727  |
| total_timesteps    | 1234325  |
| value_loss         | 0.447    |
---------------------------------
---------------------------------
| explained_variance | 0.972    |
| fps                | 2007     |
| nupdates           | 520      |
| policy_entropy     | 0.839    |
| policy_loss        | -0.126   |
| total_timesteps    | 1258575  |
| value_loss         | 0.568    |
---------------------------------
---------------------------------
| explained_variance | 0.983    |
| fps                | 2007     |
| nupdates           | 530      |
| policy_entropy     | 0.824    |
| policy_loss        | -0.00129 |
| total_timesteps    | 1282825  |
| value_loss         | 0.257    |
---------------------------------
---------------------------------
| explained_variance | 0.973    |
| fps         

---------------------------------
| explained_variance | 0.975    |
| fps                | 2017     |
| nupdates           | 780      |
| policy_entropy     | 0.89     |
| policy_loss        | -0.0497  |
| total_timesteps    | 1889075  |
| value_loss         | 0.344    |
---------------------------------
---------------------------------
| explained_variance | 0.992    |
| fps                | 2018     |
| nupdates           | 790      |
| policy_entropy     | 0.775    |
| policy_loss        | 0.111    |
| total_timesteps    | 1913325  |
| value_loss         | 0.155    |
---------------------------------
---------------------------------
| explained_variance | 0.961    |
| fps                | 2018     |
| nupdates           | 800      |
| policy_entropy     | 0.867    |
| policy_loss        | 0.0216   |
| total_timesteps    | 1937575  |
| value_loss         | 0.349    |
---------------------------------
---------------------------------
| explained_variance | 0.977    |
| fps         

---------------------------------
| explained_variance | 0.942    |
| fps                | 2026     |
| nupdates           | 1050     |
| policy_entropy     | 0.899    |
| policy_loss        | -0.0671  |
| total_timesteps    | 2543825  |
| value_loss         | 0.434    |
---------------------------------
---------------------------------
| explained_variance | 0.991    |
| fps                | 2026     |
| nupdates           | 1060     |
| policy_entropy     | 0.826    |
| policy_loss        | 0.0884   |
| total_timesteps    | 2568075  |
| value_loss         | 0.154    |
---------------------------------
---------------------------------
| explained_variance | 0.98     |
| fps                | 2027     |
| nupdates           | 1070     |
| policy_entropy     | 0.887    |
| policy_loss        | -0.1     |
| total_timesteps    | 2592325  |
| value_loss         | 0.307    |
---------------------------------
---------------------------------
| explained_variance | 0.969    |
| fps         

...

---------------------------------
| explained_variance | 0.981    |
| fps                | 2029     |
| nupdates           | 1620     |
| policy_entropy     | 0.853    |
| policy_loss        | 0.113    |
| total_timesteps    | 3926075  |
| value_loss         | 0.321    |
---------------------------------
---------------------------------
| explained_variance | 0.982    |
| fps                | 2029     |
| nupdates           | 1630     |
| policy_entropy     | 0.867    |
| policy_loss        | -0.078   |
| total_timesteps    | 3950325  |
| value_loss         | 0.327    |
---------------------------------
---------------------------------
| explained_variance | 0.975    |
| fps                | 2029     |
| nupdates           | 1640     |
| policy_entropy     | 0.866    |
| policy_loss        | 0.0149   |
| total_timesteps    | 3974575  |
| value_loss         | 0.354    |
---------------------------------
---------------------------------
| explained_variance | 0.979    |
| fps         

---------------------------------
| explained_variance | 0.99     |
| fps                | 2033     |
| nupdates           | 1890     |
| policy_entropy     | 0.998    |
| policy_loss        | 0.106    |
| total_timesteps    | 4580825  |
| value_loss         | 0.179    |
---------------------------------
---------------------------------
| explained_variance | 0.96     |
| fps                | 2034     |
| nupdates           | 1900     |
| policy_entropy     | 0.882    |
| policy_loss        | -0.0217  |
| total_timesteps    | 4605075  |
| value_loss         | 0.582    |
---------------------------------
---------------------------------
| explained_variance | 0.977    |
| fps                | 2034     |
| nupdates           | 1910     |
| policy_entropy     | 0.887    |
| policy_loss        | -0.0351  |
| total_timesteps    | 4629325  |
| value_loss         | 0.313    |
---------------------------------
---------------------------------
| explained_variance | 0.975    |
| fps         

---------------------------------
| explained_variance | 0.977    |
| fps                | 2037     |
| nupdates           | 2160     |
| policy_entropy     | 0.842    |
| policy_loss        | -0.0282  |
| total_timesteps    | 5235575  |
| value_loss         | 0.34     |
---------------------------------
---------------------------------
| explained_variance | 0.943    |
| fps                | 2037     |
| nupdates           | 2170     |
| policy_entropy     | 0.971    |
| policy_loss        | -0.381   |
| total_timesteps    | 5259825  |
| value_loss         | 1.22     |
---------------------------------
---------------------------------
| explained_variance | 0.978    |
| fps                | 2037     |
| nupdates           | 2180     |
| policy_entropy     | 0.925    |
| policy_loss        | -0.125   |
| total_timesteps    | 5284075  |
| value_loss         | 0.354    |
---------------------------------
---------------------------------
| explained_variance | 0.992    |
| fps         

---------------------------------
| explained_variance | 0.973    |
| fps                | 2040     |
| nupdates           | 2430     |
| policy_entropy     | 0.936    |
| policy_loss        | 0.0293   |
| total_timesteps    | 5890325  |
| value_loss         | 0.386    |
---------------------------------
---------------------------------
| explained_variance | 0.976    |
| fps                | 2040     |
| nupdates           | 2440     |
| policy_entropy     | 0.87     |
| policy_loss        | -0.0265  |
| total_timesteps    | 5914575  |
| value_loss         | 0.318    |
---------------------------------
---------------------------------
| explained_variance | 0.951    |
| fps                | 2040     |
| nupdates           | 2450     |
| policy_entropy     | 0.888    |
| policy_loss        | -0.146   |
| total_timesteps    | 5938825  |
| value_loss         | 0.699    |
---------------------------------
---------------------------------
| explained_variance | 0.988    |
| fps         

---------------------------------
| explained_variance | 0.974    |
| fps                | 2042     |
| nupdates           | 2700     |
| policy_entropy     | 0.903    |
| policy_loss        | -0.0472  |
| total_timesteps    | 6545075  |
| value_loss         | 0.29     |
---------------------------------
---------------------------------
| explained_variance | 0.968    |
| fps                | 2042     |
| nupdates           | 2710     |
| policy_entropy     | 0.964    |
| policy_loss        | -0.101   |
| total_timesteps    | 6569325  |
| value_loss         | 0.44     |
---------------------------------
---------------------------------
| explained_variance | 0.981    |
| fps                | 2042     |
| nupdates           | 2720     |
| policy_entropy     | 0.933    |
| policy_loss        | -0.0226  |
| total_timesteps    | 6593575  |
| value_loss         | 0.365    |
---------------------------------
---------------------------------
| explained_variance | 0.941    |
| fps         

In [ ]:
cc2=Lifecycle(env='unemployment-v1',minimal=False,mortality=mortality,perustulo=False,
              randomness=randomness,pinkslip=pinkslip,plotdebug=False)
cc2.render_distrib(load='results/distrib_base_v',n=30,stats_results='results/distrib_stats')


# Työttömyysputken poisto

Työttömyysputkelle meneminen on usein hyvin suosittua elinkaarimalleissa. Tarkastellaan millainen työllisyysvaikutus on putken poistamisella.

In [ ]:
cc1_putki=Lifecycle(env='unemployment-v1',minimal=False,include_putki=False,mortality=mortality,
                    perustulo=False,randomness=randomness)
cc1_putki.run_distrib(n=5,debug=False,steps1=size1,steps2=size2,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=batch1,batch2=batch2,
                save=perusmalli,plot=True,cont=True,start_from=perusmalli,results='results/distrib_poisto',
                callback_minsteps=callback_minsteps,rlmodel=rlmodel,twostage=twostage)


In [ ]:
cc1_putki.render_distrib(load='results/putki_results')
cc1_putki.compare_simstats('results/putki_results','results/putki_results')                    